In [9]:
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'

In [7]:
import pandas as pd
from dateutil import parser
import datetime
import numpy as np
import json
import matplotlib.pylab as plt
from tqdm import tqdm
from collections import defaultdict

import sys
sys.path.append('/srv/home/christinedk/wp_internship/collaboration/')
from features.article_history import *
from features.talk_history import *
from utils import read_revisions, np_encoder

/usr/lib/python3/dist-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.

In [2]:
cd ../data/

/srv/home/christinedk/wp_internship/data


In [3]:
ls -lh page_history/ 

total 3.5G
-rw-r--r-- 1 christinedk wikidev 1006M Mar 23 13:27 page_history-advert-meta-info.json
-rw-r--r-- 1 christinedk wikidev  323M Mar 23 13:26 page_history-autobiography-meta-info.json
-rw-r--r-- 1 christinedk wikidev  240M Mar 23 13:27 page_history-fanpov-meta-info.json
-rw-r--r-- 1 christinedk wikidev  1.1G Mar 23 13:26 page_history-peacock-meta-info.json
-rw-r--r-- 1 christinedk wikidev  910M Mar 23 13:25 page_history-weasel-meta-info.json


# Get labels

In [ ]:
for template in ['autobiography','weasel','advert','fanpov','peacock']:
    print(template)
    df = pd.read_json('/srv/home/christinedk/wp_internship/data/page_history/page_history-{}-meta-info.json'.format(template),
                 lines=True)
    df = df[df.page_namespace==0]
    df = df.sort_values(by=['page_id','revision_id'], ascending=True)
    df['has_template'] = df['has_template'].fillna(0).astype(int)
    
    tag_added = df[(df.has_template.diff()==1)&(~df.revision_is_identity_reverted)][['event_timestamp','event_user_id','page_id','page_title']]
    tag_added = tag_added[tag_added.event_user_id!=7328338]
    tag_removed = df[(df.has_template.diff()==-1)&(~df.revision_is_identity_reverted)][['event_timestamp','event_user_id','page_id','page_title']]
    tag_removed = tag_removed[tag_removed.event_user_id!=7328338]
    labels = tag_added[['event_timestamp','page_id']].drop_duplicates()
    print('num tags: ',len(labels))

    tag_added.to_csv('/srv/home/christinedk/wp_internship/data/tag_events/{}_tag_added.csv'.format(template),
                     index=False)
    tag_removed.to_csv('/srv/home/christinedk/wp_internship/data/tag_events/{}_tag_removed.csv'.format(template),
                     index=False)
    labels.to_csv('/srv/home/christinedk/wp_internship/data/labels/{}.csv'.format(template),index=False)

# Map articles to talk pages

In [8]:
for template in ['autobiography','weasel','advert','fanpov','peacock']:
    print(template)
    
    print('reading article history')
    article = pd.read_json('/srv/home/christinedk/wp_internship/data/page_history/page_history-{}-meta-info.json'.format(template),
                 lines=True)
    article = article[article.page_namespace==0]
    print(article.page_id.nunique())

    print('reading talk history')
    talk = pd.read_json('/srv/home/christinedk/wp_internship/data/page_history/talk-activity-{}-meta-info.json'.format(template),
                 lines=True)
    #extract_from_json('/srv/home/christinedk/wp_internship/data/talk_history/talk-activity-{}-meta-info.json'.format(template))
    print(talk.page_id.nunique())

    article['page_title'] = article.page_title.str.replace('_',' ').astype(str)
    talk['page_title'] = talk.page_title.str[5:]

    talk = talk[['page_id','page_title']].drop_duplicates()
    article = article[['page_id','page_title']].drop_duplicates()

    merged = talk.merge(article,on='page_title')
    merged.columns = ['talk_page_id','page_title','article_page_id']
    print('number of merged pages: {}'.format(len(merged)))
    print()

    merged.to_csv('/srv/home/christinedk/wp_internship/data/article_talk_mappings/{}.csv'.format(template),index=False)

autobiography
reading article history
3831
reading talk history
4159
number of merged pages: 3810

weasel
reading article history
1382
reading talk history
8414
number of merged pages: 1382

advert
reading article history
7235
reading talk history
44603
number of merged pages: 7167

fanpov
reading article history
828
reading talk history
1896
number of merged pages: 806

peacock
reading article history
5104
reading talk history
12943
number of merged pages: 5066



# Sample negatives

In [4]:
for template in ['autobiography','weasel','advert','fanpov','peacock']:
    print(template)

    labels = pd.read_csv('/srv/home/christinedk/wp_internship/data/labels/{}.csv'.format(template))

    df = pd.read_json('/srv/home/christinedk/wp_internship/data/page_history/page_history-{}-meta-info.json'.format(template),
                 lines=True)
    df = df[df.page_namespace==0]
    df['has_template'] = df['has_template'].fillna(0).astype(int)

    no_tag = df[(df.has_template.diff()==0)&
                (~df.revision_is_identity_reverted)][['event_timestamp','event_user_id','page_id','page_title','revision_id']]

    labels_per_page = labels.groupby('page_id').count().reset_index()
    labels_per_page = {key:value for key, value in labels_per_page.values}
    negatives = no_tag.groupby('page_id').apply(lambda x: x.sample(n=min(labels_per_page.get(x.name,0)*5,len(x))))
    negatives = negatives.reset_index(drop=True)[['event_timestamp','page_id']]
    print(len(negatives))

    negatives.to_csv('/srv/home/christinedk/wp_internship/data/negative_labels/{}.csv'.format(template),index=False)

autobiography
weasel
advert
fanpov
peacock


# Subsample talk (negative)

In [87]:
for template in ['autobiography','weasel','advert','fanpov','peacock']:
    print(template)
    talk_revisions = read_revisions(DATA_DIR+'talk_history/talk-activity-{}-meta-info.json'.format(template),
                                    rename=True).sort_values(by='event_timestamp')
    
    article_talk_mappings = pd.read_csv(DATA_DIR+'article_talk_mappings/{}.csv'.format(template),
                                   usecols=['talk_page_id','article_page_id'])
    labels = pd.read_csv(DATA_DIR+'negative_labels/{}.csv'.format(template),parse_dates=['event_timestamp'])
    labels = labels.join(article_talk_mappings.set_index('article_page_id')[['talk_page_id']],on='page_id').dropna()
    print('number of labels: ',len(labels))

    talk_pages = talk_revisions.groupby('page_id')
    last_revision = []

    for tag_date, page_id, talk_page_id in tqdm(labels.values):
        tag_talk_revisions = talk_pages.get_group(talk_page_id)
        tag_talk_revisions = tag_talk_revisions[tag_talk_revisions.event_timestamp.dt.date <= tag_date]
        if len(tag_talk_revisions) > 0:
            edit = tag_talk_revisions.iloc[-1].to_dict()
            edit['tag_date'] = tag_date
            edit['talk_page_id'] = talk_page_id
            last_revision.append(edit)
    last_revision = pd.DataFrame(last_revision).groupby('talk_page_id')
                
    last_revision_text = []
    print('scanning talk page')
    with open(DATA_DIR + 'talk_history/talk-text-{}-meta-info.json'.format(template),'rb') as f:
        for i,line in enumerate(f):
            if i%10000 == 0:
                print(i)
            snapshot=json.loads(line)
            if snapshot['page_id'] in last_revision.groups.keys():
                if snapshot['revision_id'] in last_revision.get_group(snapshot['page_id']).revision_id.values:
                    last_revision_text.append(snapshot)
    print(len(talk))
    
    with open(DATA_DIR+'talk_history/talk-subset-negative-{}.json'.format(template),'w') as f:
        json.dump(talk_dump, f, default=np_encoder)

autobiography


  0%|          | 54/21116 [00:00<00:39, 534.37it/s]

number of labels:  21116


100%|██████████| 21116/21116 [00:28<00:00, 731.83it/s]


scanning talk page
0
10000
20000
30000
40000
50000
60000
70000
80000
1120
weasel


...

scanning talk page
0
10000
20000
30000
40000
50000
60000
70000
80000
1120
peacock


  0%|          | 25/25863 [00:00<01:43, 249.02it/s]

number of labels:  25863


100%|██████████| 25863/25863 [00:36<00:00, 714.41it/s]


scanning talk page
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
1120


In [ ]:
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'

from collections import defaultdict
from dateutil import parser

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()
    
for template in ['autobiography','weasel','advert','fanpov','peacock']:
    print(template)

    article_talk_mappings = pd.read_csv(DATA_DIR+'article_talk_mappings/{}.csv'.format(template),
                                       usecols=['talk_page_id','article_page_id'])
    labels = pd.read_csv(DATA_DIR+'negative_labels/{}.csv'.format(template),parse_dates=['event_timestamp'])
    labels = labels.join(article_talk_mappings.set_index('article_page_id')[['talk_page_id']],on='page_id').dropna()
    print('number of labels: ',len(labels))
    
    page_labels = labels.groupby('talk_page_id')
    pages = labels.talk_page_id.unique()
    
    talk = {}
    with open(DATA_DIR + 'talk_history/talk-text-{}-meta-info.json'.format(template),'rb') as f:
        for line in f:
            snapshot=json.loads(line)
            page_id = snapshot['page_id']
            if page_id not in pages:
                continue

            snapshot_date = parser.parse(snapshot['revision_timestamp']).replace(tzinfo=None)
            page_lable_dates = page_labels.get_group(snapshot['page_id'])['event_timestamp']
            date_diffs = (snapshot_date - page_lable_dates).dt.days
            min_ind = date_diffs.idxmin()
            min_diff = date_diffs[min_ind]
            if 0 <= min_diff < talk.get((page_id,page_lable_dates[min_ind])[0],365):
                talk[page_id,page_lable_dates[min_ind]]=(min_diff,snapshot)

    print(len(talk))
    talk_dump = [{'talk_page_id':key[0],'event_timestamp':str(key[1]),**value[1]} for key, value in talk.items()]
    with open(DATA_DIR+'talk_history/talk-subset-negative-{}.json'.format(template),'w') as f:
        json.dump(talk_dump, f, default=np_encoder)

# Subsample talk (positive)

In [ ]:
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'

from collections import defaultdict
from dateutil import parser

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()
    
for template in ['autobiography','weasel','advert','fanpov','peacock']:
    print(template)

    article_talk_mappings = pd.read_csv(DATA_DIR+'article_talk_mappings/{}.csv'.format(template),
                                       usecols=['talk_page_id','article_page_id'])
    labels = pd.read_csv(DATA_DIR+'labels/{}.csv'.format(template),parse_dates=['event_timestamp'])
    labels = labels.join(article_talk_mappings.set_index('article_page_id')[['talk_page_id']],on='page_id').dropna()
    print('number of labels: ',len(labels))
    
    page_labels = labels.groupby('talk_page_id')
    pages = labels.talk_page_id.unique()
    
    talk = {}
    with open(DATA_DIR + 'talk_history/talk-text-{}-meta-info.json'.format(template),'rb') as f:
        for line in f:
            snapshot=json.loads(line)
            page_id = snapshot['page_id']
            if page_id not in pages:
                continue

            snapshot_date = parser.parse(snapshot['revision_timestamp']).replace(tzinfo=None)
            page_lable_dates = page_labels.get_group(snapshot['page_id'])['event_timestamp']
            date_diffs = (snapshot_date - page_lable_dates).dt.days
            min_ind = date_diffs.idxmin()
            min_diff = date_diffs[min_ind]
            if 0 <= min_diff < talk.get((page_id,page_lable_dates[min_ind])[0],365):
                talk[page_id,page_lable_dates[min_ind]]=(min_diff,snapshot)

    print(len(talk))
    talk_dump = [{'talk_page_id':key[0],'event_timestamp':str(key[1]),**value[1]} for key, value in talk.items()]
    with open(DATA_DIR+'talk_history/talk-subset-{}.json'.format(template),'w') as f:
        json.dump(talk_dump, f, default=np_encoder)